### This notebook demonstrates a full RAG pipeline:

- Downloading and transcribing YouTube audio

- Chunking and embedding the transcript

- Building a semantic search index

- Enabling strict, reference-based conversational QA and summarization

## 1. Import RAG Pipeline and Agent Utilities

In this cell, we import all the necessary functions and classes from our custom modules (`rag_pipeline.py` and `agent.py`).  
These utilities handle environment setup, audio downloading, transcription, document splitting, embedding, vector search, and agent construction for conversational QA and summarization.


In [1]:
# Import RAG pipeline utilities
from rag_pipeline import (

    load_environment,
    download_audio,
    process_audio,
    load_and_split_documents,
    create_vectorstore,
    build_qa_chain
)

# Import agent utilities
from agent import build_agent

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## 2. Download Audio from YouTube

Here, we specify the YouTube URL for the IELTS video we want to process.  
The `download_audio` function downloads the audio track of the video and saves it as an MP3 file in the output directory.  
The resulting path is displayed for confirmation.


In [2]:
youtube_url = "https://www.youtube.com/watch?v=kHTnAx6f-j0&list=PLWWR_9t3vo3OfJ62HL-nnwRaSAiLnaMSM&index=2"

In [ ]:
audio_path = download_audio(youtube_url)

In [5]:
audio_path

'output_audio\\How_to_get_a_Band_8_in_IELTS_listening.mp3'

## 3. Load Environment and Transcribe Audio

This cell loads your OpenAI API key and initializes the LLM and OpenAI client.  
Then, it processes the downloaded audio:  
- Splits it into manageable chunks  
- Transcribes each chunk using OpenAI Whisper  
- Saves the transcript to a text file  
This is the core step for converting video content into searchable text.


In [6]:
api_key, llm, clinet =load_environment()

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\rag_pipeline.py:26: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(openai_api_key=api_key, temperature=0)


In [3]:
process_audio(client=clinet)


🎧 Processing: How_to_get_a_Band_8_in_IELTS_listening
🔹 Transcribing chunk 1/3...
🔹 Transcribing chunk 2/3...
🔹 Transcribing chunk 3/3...
✅ Done. Transcript saved to: transcripts\How_to_get_a_Band_8_in_IELTS_listening.txt


## 4. Load and Split Transcript Documents

After transcription, we need to load the transcript files and split them into smaller text chunks.  
This helps with efficient embedding and retrieval, as semantic search works best on small, focused pieces of text.  
The resulting list of document chunks is displayed.


In [2]:
docs = load_and_split_documents()

In [3]:
docs

[Document(metadata={'source': 'transcripts\\How_to_get_a_Band_8_in_IELTS_listening.txt'}, page_content='# Transcript for How_to_get_a_Band_8_in_IELTS_listening'),
 Document(metadata={'source': 'transcripts\\How_to_get_a_Band_8_in_IELTS_listening.txt'}, page_content='## How_to_get_a_Band_8_in_IELTS_listening_chunk_1.mp3'),
 Document(metadata={'source': 'transcripts\\How_to_get_a_Band_8_in_IELTS_listening.txt'}, page_content="Hey everyone, Chris here from IELTSadvantage.com with another live lesson. Today's live lesson is all about listening. So we've had a huge number of people emailing us over the past few months saying that they need a band 8 in listening. It seems that many, many people want to move to Canada, they need that 8777, and sometimes what's stopping them getting there is they're getting 7, 7.5 in listening, which is a great score. But if they had that 8 or even above 8 in the listening test, they would get to Canada. Also countries like Australia, for example, has a point 

## 5. Assign Source Metadata to Document Chunks

For traceability, each chunk is annotated with metadata indicating its source (the original video or audio file).  
This is useful for providing references when answering questions and for debugging.


In [ ]:
import os
for chunk in docs:
    video_id = os.path.splitext(os.path.basename(chunk.metadata["source"]))[0]
    chunk.metadata["source"] = video_id

In [ ]:
docs

[Document(metadata={'source': 'How_to_get_a_Band_8_in_IELTS_listening'}, page_content='# Transcript for How_to_get_a_Band_8_in_IELTS_listening'),
 Document(metadata={'source': 'How_to_get_a_Band_8_in_IELTS_listening'}, page_content='## How_to_get_a_Band_8_in_IELTS_listening_chunk_1.mp3'),
 Document(metadata={'source': 'How_to_get_a_Band_8_in_IELTS_listening'}, page_content="Hey everyone, Chris here from IELTSadvantage.com with another live lesson. Today's live lesson is all about listening. So we've had a huge number of people emailing us over the past few months saying that they need a band 8 in listening. It seems that many, many people want to move to Canada, they need that 8777, and sometimes what's stopping them getting there is they're getting 7, 7.5 in listening, which is a great score. But if they had that 8 or even above 8 in the listening test, they would get to Canada. Also countries like Australia, for example, has a point system for immigration, so the higher your score, t

## 6. Generate Embeddings and Build FAISS Vector Store

Now, we generate vector embeddings for each document chunk using OpenAI embeddings.  
These embeddings are stored in a FAISS vector index, enabling fast semantic search and retrieval for question answering.


In [7]:
vectorstore = create_vectorstore(api_key=api_key, docs=docs)

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\rag_pipeline.py:227: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key, model="text-embedding-ada-002")


## 7. Create a Strict QA Prompt and Build QA Chain (Retriever)

We define a strict prompt template to ensure the model only answers questions based on the transcript, not external knowledge.  
Then, we build a RetrievalQA chain that uses the vector store and the prompt to answer questions, returning both the answer and the supporting source documents.


In [9]:
from langchain.prompts import PromptTemplate

# Custom Prompt Template for Strict QA
STRICT_QA_PROMPT = PromptTemplate(
    template="""Answer ONLY from the video transcript. If not found, say "This information is not mentioned in the transcript of the video".
    
    Transcript: {context}
    
    Question: {question}
    
    Answer:""",
    input_variables=["context", "question"]
)

In [ ]:
qa_chain = build_qa_chain(vector_store=vectorstore, chat_model=llm, return_source_documents=True)

## 8. Initialize and Test the Conversational Agent

Finally, we build a conversational agent that can answer questions and provide summaries, strictly using the transcript content.  
We test the agent with a series of queries to check that it responds appropriately, refusing to answer questions not covered in the transcript.


In [11]:
agent = build_agent(chat_model=llm , chain= qa_chain)

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\agent.py:48: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\agent.py:55: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-a

In [ ]:
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled():
    # Test 1: Remembering user info
    print(agent.invoke("Hi, I'm Layla"))  # Should store name
    print(agent.invoke("What's my name?"))  # Should return "Your name is Layla"
    
    # Test 2: Asking about the video content
    print(agent.invoke("What is the video about?"))
    print(agent.invoke("What was the main tip for listening?"))
    print(agent.invoke("What was the synonym of the word 'say'?"))
    # Returns answer from transcript with sources
    print(agent.invoke("Summarize the content of video"))
    





> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Layla! How can I assist you today?"
}
```

> Finished chain.
{'input': "Hi, I'm Layla", 'chat_history': [HumanMessage(content="Hi, I'm Layla", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Layla! How can I assist you today?', additional_kwargs={}, response_metadata={})], 'output': 'Hello Layla! How can I assist you today?', 'intermediate_steps': []}


> Entering new AgentExecutor chain...


c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "action_input": "Your name is Layla."
}
```

> Finished chain.
{'input': "What's my name?", 'chat_history': [HumanMessage(content="Hi, I'm Layla", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Layla! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Layla.', additional_kwargs={}, response_metadata={})], 'output': 'Your name is Layla.', 'intermediate_steps': []}


> Entering new AgentExecutor chain...


c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "action_input": "2 + 2 equals 4."
}
```

> Finished chain.
{'input': "What's 2+2?", 'chat_history': [HumanMessage(content="Hi, I'm Layla", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Layla! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Layla.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's 2+2?", additional_kwargs={}, response_metadata={}), AIMessage(content='2 + 2 equals 4.', additional_kwargs={}, response_metadata={})], 'output': '2 + 2 equals 4.', 'intermediate_steps': []}


> Entering new AgentExecutor chain...


...

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


```json
{
    "action": "TranscriptSummarizer",
    "action_input": "The video is about how to get a Band 8 in IELTS listening. The speaker discusses the importance of practicing strategically, learning from mistakes, and improving through practice. The video also mentions upcoming topics to be covered in future videos related to IELTS listening preparation."
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: Answer:
The video is about how to get a Band 8 in IELTS listening. The speaker discusses the importance of practicing strategically, learning from mistakes, and improving through practice.

Sources:
# Transcript for How_to_get_a_Band_8_in_IELTS_listening

think that in order to improve to a band eight in listening, you just need to do lots and lots of practice. Wrong. You're wrong. You're not going to help. Could I learn how to drive by just driving around a racetrack again and again and again and again? No. You can't just do practice tests. You have to pr

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "action_input": "The capital of Saudi Arabia is Riyadh."
}
```

> Finished chain.
{'input': 'what is the capital of saudi?', 'chat_history': [HumanMessage(content="Hi, I'm Layla", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Layla! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Layla.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's 2+2?", additional_kwargs={}, response_metadata={}), AIMessage(content='2 + 2 equals 4.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the video about?', additional_kwargs={}, response_metadata={}), AIMessage(content="Answer:\nThe video is about how to get a Band 8 in IELTS listening.\n\nSources:\nHow_to_get_a_Band_8_in_IELTS_listening: about whether you fit that bill. And if you need any oth

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\Multimodal-AI-ChatBot-for-YouTube-Video-QA\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(
